In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd


In [ ]:
missing_values = ["n/a", "na", "--","."]
dataset = pd.read_csv("openrefine/TrainingAll.csv", na_values = missing_values)

In [3]:
print(dataset.columns.values)

['File' 'NUID' 'SATREAD' 'SATMATH' 'ACTCUM' 'CARNIGIE' 'AVGSALMALE'
 'AVGSALWOM' 'AVGSALTOT' 'HOUSE' 'HDEM' 'HREP' 'SENATE' 'SDEM' 'SREP'
 'PERNWHT' 'STUFACR' 'TUITINS' 'FEEINS' 'TUITOUT' 'FEEOUT' 'FOOTBALL'
 'TUITPER' 'SLCTVTY' 'APPEAL' 'RSRCHPSTU' 'HIGHBP' 'CHGENROL' 'FRACDEGRE'
 'RETPCNT' 'BINGEDRK' 'HEALTHINS' 'MENTALHLTH' 'PHYSHLTH']


In [4]:
dataset["SATREAD"][4]

nan

In [5]:
dataset["File"].tail(1)

5814    training2016.csv
Name: File, dtype: object

In [6]:
dataset.describe()

,NUID,SATREAD,SATMATH,ACTCUM,CARNIGIE,AVGSALMALE,AVGSALWOM,AVGSALTOT,HOUSE,HDEM,...,APPEAL,RSRCHPSTU,HIGHBP,CHGENROL,FRACDEGRE,RETPCNT,BINGEDRK,HEALTHINS,MENTALHLTH,PHYSHLTH
count,5815.000000,3953.000000,3982.000000,4076.000000,5815.000000,5753.000000,5621.000000,5815.000000,5223.000000,5223.000000,...,5808.000000,5586.000000,1204.000000,5815.000000,5815.000000,5815.000000,1195.000000,1195.000000,1195.000000,1195.000000
mean,4157.673603,577.232482,584.517830,25.554711,18.493035,7611.038415,6956.482476,7262.975924,0.133001,-0.962480,...,0.778145,1269.270273,32.695847,1.000459,0.204678,74.992261,16.842929,17.032971,12.962092,13.131381
std,2373.582385,70.693090,74.392781,3.475093,5.848469,2563.514247,1961.291246,2332.068718,0.813477,0.395875,...,2.758643,4551.670027,4.831909,0.089779,0.071505,14.048816,2.738608,5.672745,1.906480,2.247178
min,10.000000,315.000000,325.000000,9.000000,-3.000000,777.000000,543.000000,748.000000,-1.308000,-1.902000,...,0.000000,0.000000,22.600000,0.142015,0.000000,0.000000,9.600000,4.100000,8.100000,7.400000
25%,2187.000000,530.000000,531.500000,23.000000,18.000000,5979.000000,5713.000000,5804.500000,-0.858000,-1.189000,...,0.229934,0.000000,29.175000,0.975602,0.164728,68.000000,14.900000,13.050000,11.700000,11.800000
50%,4119.000000,570.000000,570.000000,25.000000,19.000000,7391.000000,6766.000000,7059.000000,0.540000,-0.995000,...,0.343800,0.617348,32.300000,1.000000,0.200536,76.000000,16.600000,16.500000,13.000000,13.100000
75%,6146.000000,620.000000,620.000000,28.000000,22.000000,8934.000000,8025.000000,8482.500000,0.796000,-0.669000,...,0.594316,370.478604,35.200000,1.016784,0.236112,84.000000,18.700000,19.900000,14.100000,14.500000
max,9257.000000,800.000000,800.000000,35.000000,32.000000,22798.000000,20266.000000,22146.000000,1.228000,0.136000,...,68.600000,77304.309710,47.800000,2.653455,0.901734,100.000000,25.400000,39.500000,18.200000,20.400000


In [7]:
from sklearn import preprocessing

# create the Labelencoder object
le = preprocessing.LabelEncoder()
dataset["File"] = le.fit_transform(dataset["File"])
dataset["File"].unique()

array([0, 1, 2, 3], dtype=int64)

In [8]:
#drop the seperate the data into targets and data
target_cols = ["CHGENROL","FRACDEGRE","RETPCNT"]
cols = [col for col in dataset.columns if col not in target_cols]
data = dataset[cols]
target = dataset[target_cols]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,target,test_size = 0.20)

In [28]:
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

In [47]:
transformer_list=[('features', SimpleImputer(strategy='mean')),('indicators', MissingIndicator())]
inpute = FeatureUnion(transformer_list)
inpute = inpute.fit(X_train, y_train)

a = SimpleImputer(missing_values=np.nan, strategy='mean')
a.fit_tranform(X_train)

AttributeError: 'SimpleImputer' object has no attribute 'fit_tranform'

In [39]:
estimators = [("impute",inpute),('scale', StandardScaler()), ("clf",svm.SVR())]
pipe = Pipeline(estimators)




In [43]:

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)

param_grid = dict(clf__gamma = [gamma_range,'scale','auto'],
                  clf__C = C_range,
                  clf__kernel=['rbf','linear'])


cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

grid = GridSearchCV(pipe, param_grid=param_grid, cv=cv, n_jobs=-1, verbose=2, scoring= 'accuracy')



In [ ]:
grid.fit()